## 朴素贝叶斯

In [84]:
data = open('Data/2/train.txt','r',encoding='utf-8')
f=open('Data/2/3.csv','a',newline='',encoding='utf-8')

csv_writer = csv.writer(f)
csv_writer.writerow(["title","tag"])
m=data.readlines()


for i in m:
    n=i.strip('\n')
    L=n.split('\t')
    # print(L)
    csv_writer.writerow(L)
    
f.close()
data.close()

In [85]:
# data=pd.read_csv('Data/data_new/3.csv')
data = pd.read_csv('Data/2/3.csv', error_bad_lines=False)

data.head()
data_length = len(data)
print("数据数量：", data_length)


数据数量： 5801


## 划分训练集测试集

In [86]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['title'], data['tag'], random_state=1)

# x_train, x_test, y_train, y_test = train_test_split(data['label'], data['text_a'], random_state=1)

In [87]:
print(x_test.shape[0])
print(type(y_test))

1451
<class 'pandas.core.series.Series'>


## 分词

In [88]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]


804                   北京 联众 涉赌 36 人 被 抓   港股 联众 上午 已经 停牌
1030                   历时 9 月   利尔 化学 8.5 亿 可转债 募资 获 新进展
587                     2 名 男子 因 酒后 随地 小便 与 他人 起 争执 被 砍死
4497                               台湾 在 马来西亚 被 掳 女 游客 获释
2076    “ 尔康 和 小燕子 ” 居然 有 了 小孩 ！ 两人 的 爱情 简直 就是 部虐 狗 大戏 啊
Name: title, dtype: object

## 停词

In [89]:
infile = open("hit_stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

## 向量化

In [90]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stopwords, max_features=5000)
vectorizer.fit(x_train_fenci)

/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  % sorted(inconsistent)


CountVectorizer(max_features=5000,
                stop_words=['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→',
                            '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一',
                            './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］',
                            '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', ...])

In [91]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
#模型训练
classifier.fit(vectorizer.transform(x_train_fenci), y_train)
#使用训练好的模型进行预测
classifier.score(vectorizer.transform(fenci(x_test)), y_test)

0.8607856650585803

## 预测

In [92]:
from sklearn.metrics import classification_report
predict=classifier.predict(vectorizer.transform(fenci(x_test)))
print("每个类别的精确率与召回率",classification_report(predict, y_test))

每个类别的精确率与召回率               precision    recall  f1-score   support

           0       0.84      0.89      0.86       103
           1       0.74      0.87      0.80        85
           2       0.72      0.84      0.78       110
           3       0.90      0.82      0.86       386
           4       0.96      0.88      0.92       241
           5       0.86      0.87      0.86       330
           6       0.90      0.90      0.90       193
           7       0.23      1.00      0.38         3
         tag       0.00      0.00      0.00         0

    accuracy                           0.86      1451
   macro avg       0.68      0.79      0.71      1451
weighted avg       0.87      0.86      0.86      1451



/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [93]:
f=open('test_7.txt','r',encoding='utf-8')

L=[]
m=f.readlines()
for i in m:
    n=i.strip('\n')
    x=n.split('\t')
    L.append(x[1])

data_to_predict=pd.Series(L)
# data_to_predict=np.matrix(data_to_predict)

In [94]:
print(data_to_predict.shape)

(600,)


In [95]:
predict=classifier.predict(vectorizer.transform(fenci(data_to_predict)))

In [96]:
print(predict)

['4' '1' '5' '6' '5' '3' '6' '3' '3' '4' '3' '3' '0' '3' '3' '0' '0' '6'
 '2' '6' '5' '3' '3' '3' '4' '4' '1' '3' '5' '4' '3' '3' '0' '4' '4' '5'
 '3' '6' '6' '3' '6' '6' '1' '3' '3' '5' '1' '4' '1' '3' '0' '3' '0' '4'
 '3' '3' '4' '4' '4' '3' '4' '1' '3' '4' '4' '4' '3' '3' '3' '3' '4' '3'
 '3' '3' '2' '6' '0' '3' '5' '3' '4' '5' '4' '0' '0' '5' '5' '4' '3' '1'
 '3' '4' '1' '4' '5' '1' '4' '3' '2' '5' '5' '5' '2' '6' '3' '6' '3' '3'
 '1' '4' '1' '4' '4' '4' '0' '3' '2' '6' '5' '3' '5' '0' '3' '0' '5' '4'
 '3' '2' '2' '0' '6' '3' '2' '3' '4' '2' '4' '6' '4' '5' '2' '3' '5' '3'
 '6' '3' '6' '4' '4' '3' '3' '5' '6' '4' '5' '0' '3' '3' '4' '3' '4' '4'
 '3' '5' '1' '5' '5' '4' '2' '3' '0' '3' '4' '3' '5' '3' '6' '0' '6' '4'
 '1' '3' '2' '4' '5' '0' '4' '3' '1' '6' '3' '3' '0' '0' '3' '3' '3' '3'
 '3' '5' '3' '6' '6' '2' '3' '6' '1' '1' '1' '1' '5' '3' '1' '3' '4' '6'
 '1' '3' '3' '3' '3' '6' '3' '1' '3' '4' '4' '6' '5' '0' '4' '1' '6' '4'
 '5' '3' '0' '3' '3' '1' '0' '0' '5' '1' '6' '4' '2

In [97]:
import jieba

def fenci(news_title):
    words = ' '.join(jieba.cut(news_title))
    return words

f_r = open("test_6.txt", "r", encoding="utf-8")
f_w = open("result_6_zhw.txt", "w", encoding="utf-8")

m = f_r.readlines()
for i in m:
    n = i.strip('\n')
    L = n.split('\t')
    data_id = L[0]
    news_title = L[1]
    prediction = classifier.predict(vectorizer.transform([fenci(news_title)]))
    line = data_id + '\t' + str(prediction[0]) + '\n'
    f_w.write(line)

f_r.close()
f_w.close()


## TF-IDF预处理

In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
#使用tf-idf把文本转为向量
tv = TfidfVectorizer(stop_words=stopwords, max_features=5000, lowercase = False)
tv.fit(x_train_fenci)
#模型训练
classifier.fit(tv.transform(fenci(x_train)), y_train)
#利用训练好的模型测试
# predict=classifier.predict(tv.transform(fenci(x_test)))
classifier.score(tv.transform(fenci(x_test)), y_test)

# print("每个类别的精确率与召回率",classification_report(predict, y_test))

/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  % sorted(inconsistent)


AttributeError: 'Series' object has no attribute 'decode'

In [ ]:
predict=classifier.predict(tv.transform(fenci(x_test)))

In [ ]:
print("每个类别的精确率与召回率",classification_report(predict, y_test))

每个类别的精确率与召回率               precision    recall  f1-score   support

           0       0.62      0.78      0.69      2322
           1       0.61      0.84      0.71      2106
           2       0.80      0.77      0.79      3256
           3       0.91      0.79      0.85     10586
           4       0.90      0.85      0.87      5816
           5       0.85      0.82      0.84      9142
           6       0.85      0.93      0.89      4876
           7       0.02      0.50      0.03        10

    accuracy                           0.83     38114
   macro avg       0.69      0.79      0.71     38114
weighted avg       0.84      0.83      0.83     38114



## N-gram 模型

In [ ]:
tv_2gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(1,2),lowercase = False)
tv_2gram.fit(x_train_fenci)
#训练模型
clf_2gram = MultinomialNB()
clf_2gram.fit(tv_2gram.transform(fenci(x_train)), y_train)
#预测
clf_2gram.score(tv_2gram.transform(fenci(x_test)), y_test)

/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  % sorted(inconsistent)


0.8230046701999265

In [ ]:
predict=clf_2gram.predict(tv_2gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率",classification_report(predict, y_test))

每个类别的精确率与召回率               precision    recall  f1-score   support

           0       0.60      0.79      0.68      2224
           1       0.61      0.84      0.70      2111
           2       0.81      0.77      0.79      3297
           3       0.91      0.78      0.84     10680
           4       0.89      0.85      0.87      5800
           5       0.85      0.82      0.84      9104
           6       0.85      0.92      0.89      4889
           7       0.02      0.56      0.03         9

    accuracy                           0.82     38114
   macro avg       0.69      0.79      0.71     38114
weighted avg       0.84      0.82      0.83     38114



## 3-gram模型

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv_3gram = TfidfVectorizer(stop_words=stopwords, max_features=5000, ngram_range=(2,3),lowercase = False)
tv_3gram.fit(x_train_fenci)
#训练模型
clf_3gram = MultinomialNB()
clf_3gram.fit(tv_3gram.transform(fenci(x_train)), y_train)
#预测
clf_3gram.score(tv_3gram.transform(fenci(x_test)), y_test)

/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①Ａ', '①Ｂ', '①Ｃ', '①Ｄ', '①Ｅ', '①ａ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②Ｂ', '②Ｇ', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③Ｆ', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２'] not in stop_words.
  % sorted(inconsistent)


0.44167497507477566

In [ ]:
predict=clf_3gram.predict(tv_3gram.transform(fenci(x_test)))
print("每个类别的精确率与召回率",classification_report(predict, y_test))

每个类别的精确率与召回率               precision    recall  f1-score   support

           0       0.29      0.65      0.40      2543
           1       0.33      0.72      0.45      1985
           2       0.89      0.33      0.48     17272
           3       0.33      0.60      0.43      2340
           4       0.20      0.85      0.32       307
           5       0.06      0.66      0.11       125
           6       0.11      0.88      0.19       112
           7       0.00      0.00      0.00         0

    accuracy                           0.43     24684
   macro avg       0.28      0.59      0.30     24684
weighted avg       0.71      0.43      0.46     24684



/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhanghongwei/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
